In [7]:
import os
from dotenv import load_dotenv
from typing import List
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr

load_dotenv(override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "mydummykey")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY", "mydummykey")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", "mydummykey")

In [7]:
def shout(txt):
    return txt.upper()

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6, placeholder="type away..")],
    outputs=[gr.Textbox(label="Response:", lines=6, placeholder="llm speaks..")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()

system_message = "You are a helpful assistant who responds only in Markdown."

def stream_gpt(prompt):
    result = ""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )    
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


def stream_claude(prompt):
    response = ""
    with claude.messages.stream(
        max_tokens=1024,
            messages=[{"role": "user", "content": prompt}],
            model="claude-3-haiku-20240307",
            system=system_message,
            temperature=0.7
    ) as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
    

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result  # same as for item in result yield item
    
    
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="Your message:"),
        gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


### Conversational interface on gradio


In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

system_message = "You are a helpful phisics tutor, who specialises in intuitive explanations. \
    All your responces are in markdown. You allways end the question by suggesting next steps."

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] \
    + history \
    + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response


gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful phisics tutor, who specialises in intuitive explanations.     All your responces are in markdown. You allways end the question by suggesting next steps.'}, {'role': 'user', 'content': 'hi, what is gradient descent'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'hi, what is gradient descent', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "### What is Gradient Descent?\n\nGradient descent is a mathematical optimization technique used for finding the minimum of a function. It’s commonly used in machine learning and artificial intelligence to optimize models.\n\n#### Intuitive Explanation:\n\nThink of gradient descent like hiking down a mountain. Your goal is to find the lowest point (the valley) on the terrain (the function).\n\n1. **Starting Point**: You begin at a random position on the mountain.\n  \n2. **Slope**: At each step, you check the slope of the hill at your 

In [22]:
def echo(message, history):
    print(history)
    return message

demo = gr.ChatInterface(fn=echo, 
                        type="messages",  
                        title="Echo Bot")
demo.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'hi', 'options': None}]
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'two', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'two', 'options': None}]


In [ ]:
def yes(message, history):
    return "yes"

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " ) #+ data.value["value"]
    else:
        print("You downvoted this response: ")

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type='messages',placeholder="<strong>Your Personal Yes-Man</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(fn=yes, type="messages", chatbot=chatbot)
    
demo.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


You upvoted this response: 


## code assistant with locally running models


In [ ]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = 'deepseek-r1:8b'

system_message = "You are a helpful AI coding assistant called Bharath. \
    Give short, courteous answers, no more than a few sentences. \
    Allways be accurate, if you don't know the answer, say so."

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] \
    + history \
    + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        print(response)
        yield response


gr.ChatInterface(fn=chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


<think>
<think>

<think>
Okay
<think>
Okay,
<think>
Okay, so
<think>
Okay, so I
<think>
Okay, so I'm
<think>
Okay, so I'm trying
<think>
Okay, so I'm trying to
<think>
Okay, so I'm trying to figure
<think>
Okay, so I'm trying to figure out
<think>
Okay, so I'm trying to figure out how
<think>
Okay, so I'm trying to figure out how to
<think>
Okay, so I'm trying to figure out how to solve
<think>
Okay, so I'm trying to figure out how to solve for
<think>
Okay, so I'm trying to figure out how to solve for 
<think>
Okay, so I'm trying to figure out how to solve for 3
<think>
Okay, so I'm trying to figure out how to solve for 3/
<think>
Okay, so I'm trying to figure out how to solve for 3/7
<think>
Okay, so I'm trying to figure out how to solve for 3/7 using
<think>
Okay, so I'm trying to figure out how to solve for 3/7 using algebra
<think>
Okay, so I'm trying to figure out how to solve for 3/7 using algebra tiles
<think>
Okay, so I'm trying to figure out how to solve for 3/7 using algebra

In [8]:
client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Say 'double bubble bath' three times fast.",
        },
    ],
    stream=True,
)

response = ""
for chunk in stream:
    response += chunk
    print(response)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: mydummykey. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}